In [12]:
import sys
import os
import requests
import threading
import time
import json
import re
import pandas
import collections
from bs4 import BeautifulSoup as soup
from datetime import datetime
import html as h

class SCRAPER:

	URL = "https://www.sec.gov/cgi-bin/browse-edgar?CIK=%s&start=%d&count=100"
	LOCK = threading.Semaphore( value=1 )
	THREADS = 0
	CCOUNT = 0
	ERRORS = 0
	FILECOUNT = 0
	FILES = True
	LINKS = []
	LOOPER = []
	DATA = {}
	HEADERS = {
		'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:64.0) Gecko/20100101 Firefox/64.0',
	}

	def __init__(self, _ticker, _howmany, _threads=5, _verbose=False):
		''' A simple and straightforward Scraper! '''
		self.ticker = _ticker
		self.count  = _howmany
		self.max    = _threads   ## More threads more speed except for the file accumulation process  ##
		self.verbose = _verbose  ## Verbose Mode. Print Messages!  ##

	def table(self, _html):
		links, html = [], soup( _html, features="lxml" )
		table = html.find( "table", attrs={ 'class': 'tableFile2' } )

		## Extracting Data from tables ##
		if table:
			rows = table.findChildren( "tr" )[1:]
			for row in rows:
				( file, format, desc, date, number ) = row.findChildren( "td" )
				if file.text == "10-Q" or file.text == "10-q" or file.text == "10q" or file.text == "10Q":
					intdata = format.findChild( "a", attrs={ 'id': 'interactiveDataBtn' } )
					if intdata:
						links.append( (date.text, "https://www.sec.gov%s" % intdata.get( "href" )) )
					else:
						if self.verbose:
							print("[!] Received 10Q file without interactive data link. ")
			return links
		else:
			self.FILES = False ## Stop if no table is detected ##

	def request(self):
		url = self.URL % ( self.ticker, self.CCOUNT )
		for n in range( 3 ):                             ##  Retry three times in case an error occurs  ##
			try:
				r = requests.get( url, headers=self.HEADERS )
				if r.status_code == 200:
					self.CCOUNT += 100
					q10links = self.table( r.text )
					self.LINKS += q10links
					if self.verbose:
						print("[*] Received Number of Files: %d Requested Counter: %d" % (len( q10links ), self.CCOUNT))
					self.FILECOUNT += len( q10links )
					if self.FILECOUNT >= self.count:
						self.FILES = False
				else:
					raise Exception( "[~] Invalid Response Code Received : %d" % r.status_code )
				break
			except KeyboardInterrupt:
				raise KeyboardInterrupt()
			except:
				if self.verbose:
					print("[!] Failed connection to Server. Check Your Connection. Trying Again!")

	def search(self):
		if self.verbose:
			print("[^] Requesting 10Q files from the Server!")
		while self.FILES:
			self.request(  )
		if self.verbose:
			print("[*] Acquired Number of 10Q files: %d" % ( self.count ))

	def pprequest( self, _link, _type ):
		''' The main function to extract all financial statements data. It accepts the link to financial document and return it's dataframe as of created by pandas
			DataFrame() object. It is highly flexbile and determines whether there are any colspans in the table or not. If there are, they will be extracted and
			according deleted when the next colspan is encoundered. whether the argument _link is the link to the financial document and _type is the document type.
			_type argument is only for testing purposes. '''
		try:
			r = requests.get( _link, headers=self.HEADERS )
		except:
			if self.verbose:
				print("[!] Error Requesting Financial Statement. Type: %s" % _type)
			return pandas.DataFrame()
		if r.status_code == 200:
			html = soup( r.text, features="lxml" )
			table = html.find( "table", attrs={ 'class': 'report' } )
			if table:
				rows, differ, data = table.findChildren( "tr" ), 0, collections.OrderedDict()
				for row in rows:
					if row.findChild( "th" ):
						differ += 1
				headerslist = rows[ :differ ]
				tabdatalist = rows[ differ: ]
				for headers in headerslist:
					headerslist[ headerslist.index( headers ) ] = headers.findChildren( "th" )
				for tabdata in tabdatalist:
					tabdatalist[ tabdatalist.index( tabdata ) ] = tabdata.findChildren( "td" )
				for headers in headerslist:
					for header in headers:
						if header.get( "colspan" ) and int(header.get( "colspan" )) > 1:
							colspan = int( header.get( "colspan" ) )
							for col in range( colspan ):
								try:
									data[ h.unescape( header.text ) + " - " + h.unescape( headerslist[ headerslist.index( headers ) + 1 ][ col ].text ).strip( "\n" ) ] = []
								except IndexError:
									data[ h.unescape( header.text ).strip( "\n" ) ] = []
							try:
								del headerslist[ headerslist.index( headers ) + 1 ][ :colspan ]
							except:
								pass
							del colspan
						else: 
							data[ h.unescape( header.text ) ] = []
				indexer = list(data.items())
				for ( _key, _value ) in indexer:
					for tabdata in tabdatalist:
						try:
							try:
								data[ _key ].append( int(float(h.unescape( tabdata[ indexer.index( ( _key, _value ) ) ].text ).strip( "\n" ).replace(" ", "").replace("$", "").replace(",", "").replace("(", "").replace(")", ""))) )
							except ValueError:
								data[ _key ].append( h.unescape( tabdata[ indexer.index( ( _key, _value ) ) ].text ).strip( "\n" ) )
						except IndexError:
							data[ _key ].append( "" )
				pd = pandas.DataFrame( data, index=range( len( tabdatalist ) ) )
				pd = pd.set_index( list( pd )[0] )
				return pd
			else:	
				return pandas.DataFrame()

	def prequest( self, _date, _link ):
		self.THREADS += 1

		if self.verbose:
			self.LOCK.acquire()
			print("[^] Requesting Q10 File! Date: %s" % (_date))
			self.LOCK.release()

		for n in range(2):
			r = requests.get( _link )
			if r.status_code == 200:
				html = soup( r.text, features="lxml" )
				r1link  = re.search( r"reports\[0\+1\] = \"(.*?)\";", r.text )
				r2link  = re.search( r"reports\[1\+1\] = \"(.*?)\";", r.text )
				r3link  = re.search( r"reports\[2\+1\] = \"(.*?)\";", r.text )
				r4link  = re.search( r"reports\[3\+1\] = \"(.*?)\";", r.text )
				r5link  = re.search( r"reports\[4\+1\] = \"(.*?)\";", r.text )
				r6link  = re.search( r"reports\[5\+1\] = \"(.*?)\";", r.text )
				r7link  = re.search( r"reports\[6\+1\] = \"(.*?)\";", r.text )
				r8link  = re.search( r"reports\[7\+1\] = \"(.*?)\";", r.text )
				r9link  = re.search( r"reports\[8\+1\] = \"(.*?)\";", r.text )

				r1id    = html.find( "li", attrs={ 'id': 'r1' } )
				r2id    = html.find( "li", attrs={ 'id': 'r2' } )
				r3id    = html.find( "li", attrs={ 'id': 'r3' } )
				r4id    = html.find( "li", attrs={ 'id': 'r4' } )
				r5id    = html.find( "li", attrs={ 'id': 'r5' } )
				r6id    = html.find( "li", attrs={ 'id': 'r6' } )
				r7id    = html.find( "li", attrs={ 'id': 'r7' } )
				r8id    = html.find( "li", attrs={ 'id': 'r8' } )
				r9id    = html.find( "li", attrs={ 'id': 'r9' } )

				rlinks = (r1link, r2link, r3link, r4link, r5link, r6link, r7link, r8link, r9link)
				rids = (r1id, r2id, r3id, r4id, r5id, r6id, r7id, r8id, r9id)
				income = blance = caflow = pandas.DataFrame()

				for rid in rids:
					if (income.empty) and (rid):
						if ( (re.search("income", rid.text, re.I)) or (re.search("operations", rid.text, re.I)) ) and ( (not re.search("comprehensive", rid.text, re.I)) or (not re.search("parenthetical", rid.text, re.I)) ):
							income = self.pprequest( "https://www.sec.gov" + rlinks[ rids.index( rid ) ].groups()[0], "Income" )
					if (blance.empty) and (rid):
						if ( (re.search("balance sheet", rid.text, re.I)) or (re.search("financial position", rid.text, re.I )) ) and ( (not re.search("comprehensive", rid.text, re.I)) or (not re.search("parenthetical", rid.text, re.I)) ):
							blance = self.pprequest( "https://www.sec.gov" + rlinks[ rids.index( rid ) ].groups()[0], "Balance" )
					if (caflow.empty) and (rid):
						if ( (re.search("CONSOLIDATED STATEMENTS OF CASH FLOWS", rid.text, re.I)) ) and ( (not re.search("comprehensive", rid.text, re.I)) or (not re.search("parenthetical", rid.text, re.I)) ):
							caflow = self.pprequest( "https://www.sec.gov" + rlinks[ rids.index( rid ) ].groups()[0], "Cash Flow" )

				if self.verbose:
					self.LOCK.acquire()
					print("[*] Received Q10 File. Date: %s" % _date)
					self.LOCK.release()

				self.DATA[ datetime.strptime( _date, "%Y-%m-%d" ) ] = ( income, blance, caflow )
			else:
				if self.verbose:
					print("[!] Invalid Response Code Received for link: %s" % _link)
			break

		self.THREADS -= 1

	def sort(self):
		''' This is an object to sort the found files according to their publishing date '''
		dateobjs = []
		for ( date, link ) in self.LINKS:
			dateobjs.append( datetime.strptime( date, "%Y-%m-%d" ) )
		dateobjs = sorted( dateobjs, reverse=True )[ :self.count ]
		for dateobj in dateobjs:
			for ( date, link ) in self.LINKS:
				if date == dateobj.strftime( "%Y-%m-%d" ):
					self.LOOPER.append( ( date, link ) )

	def scrape(self):
		for (date, link) in set(self.LOOPER):
			_t = threading.Thread( target=self.prequest, args=( date, link ) )
			_t.daemon = True
			_t.start()

			while self.THREADS >= self.max:
				time.sleep( 1 )

			time.sleep( 1 )

		while self.THREADS > 0:
			time.sleep( 1 )
		if self.verbose:
			print("[<] Done!")
		return self.DATA

	def acquire(self):
		''' Function to execute other three functions: search(), sort() and scrape() in sequence '''
		self.search()
		self.sort()
		self.scrape()

		return self.DATA

In [13]:
scraper = SCRAPER( "MSFT", 3, _verbose=True, _threads=5 )
data = scraper.acquire()

[^] Requesting 10Q files from the Server!
[*] Received Number of Files: 2 Requested Counter: 100
[*] Received Number of Files: 2 Requested Counter: 200
[*] Acquired Number of 10Q files: 3
[^] Requesting Q10 File! Date: 2018-10-24
[^] Requesting Q10 File! Date: 2018-04-26
[^] Requesting Q10 File! Date: 2019-01-30
[*] Received Q10 File. Date: 2018-10-24
[*] Received Q10 File. Date: 2018-04-26
[*] Received Q10 File. Date: 2019-01-30
[<] Done!


In [5]:
import datetime
dateobjs = list(data.keys())
dateobjs

[datetime.datetime(2018, 10, 24, 0, 0),
 datetime.datetime(2018, 4, 26, 0, 0),
 datetime.datetime(2016, 10, 20, 0, 0),
 datetime.datetime(2018, 1, 31, 0, 0),
 datetime.datetime(2016, 1, 28, 0, 0),
 datetime.datetime(2017, 1, 26, 0, 0),
 datetime.datetime(2019, 1, 30, 0, 0),
 datetime.datetime(2016, 4, 21, 0, 0),
 datetime.datetime(2017, 10, 26, 0, 0),
 datetime.datetime(2017, 4, 27, 0, 0)]

In [10]:
dateobjs = sorted( dateobjs, reverse=True )
dateobjs

[datetime.datetime(2019, 1, 30, 0, 0),
 datetime.datetime(2018, 10, 24, 0, 0),
 datetime.datetime(2018, 4, 26, 0, 0),
 datetime.datetime(2018, 1, 31, 0, 0),
 datetime.datetime(2017, 10, 26, 0, 0),
 datetime.datetime(2017, 4, 27, 0, 0),
 datetime.datetime(2017, 1, 26, 0, 0),
 datetime.datetime(2016, 10, 20, 0, 0),
 datetime.datetime(2016, 4, 21, 0, 0),
 datetime.datetime(2016, 1, 28, 0, 0)]

In [11]:
for dateobj in dateobjs:
    datestring = dateobj.strftime( "%Y-%m-%d" )
    ( income, blance, cashflow ) = data[ datestring ]

KeyError: '2019-01-30'